In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models, transforms
from dataset_eval import Dataset
from torch.utils.data import DataLoader

import os
import cv2
import numpy as np

from time import time
from visdom import Visdom
import copy

num_classes = 7
batch_size = 32
feature_extract = False
pretrained = False
input_size = 224

In [2]:
def eval_model(model, dataloaders):
    model.eval() 
    
    running_loss = 0.0
    running_corrects = 0

    # Iterate over data.
    for step, (inputs, labels) in enumerate(dataloaders):
        inputs = inputs.cuda()
        labels = labels.cuda()

        outputs = model(inputs)
        # loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)

        # statistics
        running_corrects += torch.sum(preds == labels)

    return running_corrects

In [6]:
BASE_DIR = '/home/jun/Github/BoVW/dataset/'

train_ds = Dataset(BASE_DIR)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

model = models.vgg11_bn(pretrained=pretrained).cuda()
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs,num_classes)
model =  torch.nn.DataParallel(model).cuda()
model.load_state_dict(torch.load('./model/net25-0.001.pth'))
# model.load_state_dict(torch.load('./model/net_finetune21-0.000.pth'))
# model.load_state_dict(torch.load('./model/net_scratch32-0.000.pth'))

loss_fn = torch.nn.CrossEntropyLoss()
acc = eval_model(model, train_dl)
print(acc)


['sea', 'green', 'city', 'house_building', 'face', 'house_indoor', 'office']
tensor(191, device='cuda:0')


In [4]:

# test_img = cv2.imread(BASE_DIR + 'train/2/1.jpg')
# test_img = cv2.resize(test_img, (input_size,input_size), interpolation=cv2.INTER_CUBIC)
# test_img = np.moveaxis(test_img,2,0)
# test_img = torch.FloatTensor(test_img).cuda().unsqueeze(0)
# # print(test_img.shape)

# outputs = model(test_img)
# print(outputs.shape)
# print(outputs)
# # loss = loss_fn(outputs, labels)
# _, preds = torch.max(outputs, 1)
# print(preds)